In [1]:
#import dependancies
import pandas as pd
import numpy as np
import requests 
import time 
import json

from census_key import api_key

In [2]:
markets_df = pd.read_csv('Markets_for_api.csv', encoding = "ISO-8859-1", dtype=str)

In [3]:
markets_df.head(20)

,Market_id,Market_name,Market_state,State_code,FIPS_code,Lat,Long
0,1,Albany,NY,36,36001,42.6526,\t-73.7562
1,2,Atlanta,GA,13,13121,\t33.753746,\t-84.386330
2,3,Baltimore,MD,24,24510,\t39.299236,\t-76.609383
3,4,Boise,ID,16,16001,\t43.618881,\t-116.215019
4,5,Boston,MA,25,25025,\t42.361145,\t-71.057083
5,6,Buffalo,NY,36,36029,\t42.880230,\t-78.878738
6,7,Charlotte,NC,37,37119,\t35.227085,\t-80.843124
7,8,Chicago,IL,17,17031,\t41.881832,\t-87.623177
8,9,Cincinnati,OH,39,39061,\t39.103119,\t-84.512016
9,10,Columbus,OH,39,39049,\t39.983334,\t-82.983330


In [4]:
markets_df["County_code"]=""

for index, row in markets_df.iterrows():
    FIPS = markets_df.loc[index,"FIPS_code"]
    county_code = FIPS[2:5]
    markets_df.loc[index,"County_code"] = county_code
    
#markets_df.head()
    

In [5]:
##Create holders for Total Population, White Alone Population, Age 20-24, Age 25 - 29, Age 30 - 34, Age 35 - 39 and Median Income
markets_df["Total_population"]=""
markets_df["Non_White_pop"]=""
markets_df["Diversity%"]=""
markets_df["Millenial_pop"]=""
markets_df["Millenial%"]=""
markets_df["Median_income"]=""

#markets_df.tail()
#There are 42 markets, just FYI

In [6]:
print("Log Start")
print("--------------------------")

for index, row in markets_df.iterrows():
    
    #set needed variables
    county = markets_df.loc[index, "County_code"]
    state = markets_df.loc[index, "State_code"]
    data = "B01001_001E,B01001A_001E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B06011_001E"
        
    query_url = "http://api.census.gov/data/2016/acs/acs1?get=" + str(data) + "&for=county:" + str(county) + "&in=state:" + str(state) + "&key=" + str(api_key)
    
    try:
        response = requests.get(query_url).json()
    
        census_data = response[1]

        for i in range(0, len(census_data)): 
            census_data[i] = int(census_data[i]) 

        print(markets_df.loc[index,"Market_id"])
        
        try:
            #sum all age data
            all_females = sum(census_data[2:8])
            all_males = sum(census_data[8:14])
            millenials = all_females + all_males
            
            total_pop = census_data[0]
            white_pop = census_data[1]
            
            non_white = total_pop - white_pop
            #print(total_pop, non_white)
            
            diversity = non_white/total_pop
            #print(diversity)

        except:
            print('error with conversion')
        
        try:
            markets_df.loc[index, "Total_population"] = total_pop
            markets_df.loc[index, "Non_White_pop"]= non_white
            markets_df.loc[index,"Diversity%"]= diversity
            markets_df.loc[index,"Millenial_pop"]= millenials
            markets_df.loc[index,"Millenial%"]= millenials/total_pop
            markets_df.loc[index,"Median_income"]=census_data[14]
        
        except:
            print('error with api call')
            
    except:
        print(query_url)


print("---------------")
print("Log End")

Log Start
--------------------------
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
http://api.census.gov/data/2016/acs/acs1?get=B01001_001E,B01001A_001E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B06011_001E&for=county:&in=state:0&key=5b32564c5f4ebf42969c99f2ac96affff18bb411
---------------
Log End


In [7]:
#Market_id 43 (last one) fails because it's the total u.s., so there's no FIPS county or state to call. 
#Build a custom call to grab total us data. Region seems to be as aggregate as we can get, so we'll need to sum

query_url = "http://api.census.gov/data/2016/acs/acs1?get=" + str(data) + "&for=region:*&key=" + str(api_key)


response = requests.get(query_url).json()
        
northeast = response[1]
midwest = response[2]
south = response[3]
west = response[4]

try:
    for i in range(0,(len(northeast)-1)):
        northeast[i] = int(northeast[i])
                       
    for i in range(0,(len(midwest)-1)):
        midwest[i] = int(midwest[i])
                       
    for i in range(0,(len(south)-1)):
        south[i] = int(south[i])
                       
    for i in range(0,(len(west)-1)):
        west[i] = int(west[i])
        
except:
    print("couldn't convert")


total_pop = northeast[0] + midwest[0] + south[0]+ west[0]
print(total_pop)

all_females = sum(northeast[2:8])+ sum(midwest[2:8]) + sum(south[2:8]) + sum(west[2:8])
all_males = sum(northeast[8:14])+sum(midwest[8:14])+sum(south[8:14])+sum(west[8:14])
millenials = all_females + all_males
print(millenials)

white_pop = northeast[1] + midwest[1] + south[1]+ west[1]
non_white = total_pop - white_pop
diversity = non_white/total_pop

#taking the average of the median
median_inc = (northeast[14] + midwest[14] + south[14] + west[14])/4

markets_df.loc[42, "Total_population"] = total_pop
markets_df.loc[42, "Non_White_pop"]= non_white
markets_df.loc[42,"Diversity%"]= diversity
markets_df.loc[42,"Millenial_pop"]= millenials
markets_df.loc[42,"Millenial%"]= millenials/total_pop
markets_df.loc[42,"Median_income"]= median_inc

323127515
87413147


In [8]:
markets_df

,Market_id,Market_name,Market_state,State_code,FIPS_code,Lat,Long,County_code,Total_population,Non_White_pop,Diversity%,Millenial_pop,Millenial%,Median_income
0,1,Albany,NY,36,36001,42.6526,\t-73.7562,001,308846,78015,0.252602,91345,0.295762,31505
1,2,Atlanta,GA,13,13121,\t33.753746,\t-84.386330,121,1023336,567804,0.554856,319104,0.311827,34145
2,3,Baltimore,MD,24,24510,\t39.299236,\t-76.609383,510,614664,425404,0.692092,205273,0.33396,27617
3,4,Boise,ID,16,16001,\t43.618881,\t-116.215019,001,444028,41177,0.0927351,123537,0.278219,30690
4,5,Boston,MA,25,25025,\t42.361145,\t-71.057083,025,784230,352165,0.449058,309286,0.394382,31657
5,6,Buffalo,NY,36,36029,\t42.880230,\t-78.878738,029,921046,202076,0.219398,247575,0.268798,30187
6,7,Charlotte,NC,37,37119,\t35.227085,\t-80.843124,119,1054835,479904,0.454956,324562,0.30769,32763
7,8,Chicago,IL,17,17031,\t41.881832,\t-87.623177,031,5203499,2284040,0.438943,1565547,0.300864,30986
8,9,Cincinnati,OH,39,39061,\t39.103119,\t-84.512016,061,809099,266221,0.329034,220920,0.273044,30319
9,10,Columbus,OH,39,39049,\t39.983334,\t-82.983330,049,1264518,409284,0.323668,412408,0.326138,30768


In [9]:
markets_df.to_csv("Markets_demo_2016.csv")